In [2]:
%load_ext autoreload

%autoreload 2

In [3]:
from ark import io
from pathlib import Path
import spatialdata as sd
from spatialdata.models import C
from spatial_image import SpatialImage

## 1. Convert a OG cohort to a spatial data cohort

Set the path of the cohort

In [4]:
cohort_dir = Path("../data/example_dataset/image_data")

Load the cohort

In [7]:
sdata: sd.SpatialData = io.load_cohort(cohort_dir)

View the cohort

In [8]:
sdata

SpatialData object with:
└── Images
      ├── 'fov0': SpatialImage[cyx] (22, 512, 512)
      ├── 'fov1': SpatialImage[cyx] (22, 1024, 1024)
      ├── 'fov2': SpatialImage[cyx] (22, 1024, 1024)
      ├── 'fov3': SpatialImage[cyx] (22, 1024, 1024)
      ├── 'fov4': SpatialImage[cyx] (22, 1024, 1024)
      ├── 'fov5': SpatialImage[cyx] (22, 1024, 1024)
      ├── 'fov6': SpatialImage[cyx] (22, 1024, 1024)
      ├── 'fov7': SpatialImage[cyx] (22, 1024, 1024)
      ├── 'fov8': SpatialImage[cyx] (22, 1024, 1024)
      ├── 'fov9': SpatialImage[cyx] (22, 1024, 1024)
      └── 'fov10': SpatialImage[cyx] (22, 1024, 1024)
with coordinate systems:
▸ 'global', with elements:
        fov0 (Images), fov1 (Images), fov2 (Images), fov3 (Images), fov4 (Images), fov5 (Images), fov6 (Images), fov7 (Images), fov8 (Images), fov9 (Images), fov10 (Images)

We can access any Image in the Spatial Data object by using the image name as a key in the `images` dictionary.

In [9]:
sdata.images.keys()

dict_keys(['fov4', 'fov3', 'fov10', 'fov2', 'fov5', 'fov0', 'fov7', 'fov9', 'fov8', 'fov6', 'fov1'])

In [10]:
sdata.images["fov0"]

<xarray.SpatialImage 'image' (c: 22, y: 512, x: 512)>
dask.array<_map_read_frame, shape=(22, 512, 512), dtype=float32, chunksize=(1, 512, 512), chunktype=numpy.ndarray>
Coordinates:
  * c        (c) <U11 'CD3' 'CD4' 'CD8' 'CD14' ... 'Ki67' 'PD1' 'SMA' 'Vim'
  * y        (y) float64 0.5 1.5 2.5 3.5 4.5 ... 507.5 508.5 509.5 510.5 511.5
  * x        (x) float64 0.5 1.5 2.5 3.5 4.5 ... 507.5 508.5 509.5 510.5 511.5
Attributes:
    transform:  {'global': Identity }

We can look at the channels in our cohort

In [11]:
sdata.images["fov0"].c

<xarray.SpatialImage 'c' (c: 22)>
array(['CD3', 'CD4', 'CD8', 'CD14', 'CD20', 'CD31', 'CD45', 'CD68', 'CD163',
       'CK17', 'Collagen1', 'ECAD', 'Fibronectin', 'GLUT1', 'H3K9ac',
       'H3K27me3', 'HLADR', 'IDO', 'Ki67', 'PD1', 'SMA', 'Vim'], dtype='<U11')
Coordinates:
  * c        (c) <U11 'CD3' 'CD4' 'CD8' 'CD14' ... 'Ki67' 'PD1' 'SMA' 'Vim'
Attributes:
    long_name:  c
    units:

Select a few channels from `fov0`

In [12]:
sdata.images["fov0"].sel({C: ["CD3", "CD4", "CD8"]})

<xarray.SpatialImage 'image' (c: 3, y: 512, x: 512)>
dask.array<getitem, shape=(3, 512, 512), dtype=float32, chunksize=(1, 512, 512), chunktype=numpy.ndarray>
Coordinates:
  * c        (c) <U11 'CD3' 'CD4' 'CD8'
  * y        (y) float64 0.5 1.5 2.5 3.5 4.5 ... 507.5 508.5 509.5 510.5 511.5
  * x        (x) float64 0.5 1.5 2.5 3.5 4.5 ... 507.5 508.5 509.5 510.5 511.5
Attributes:
    transform:  {'global': Identity }

We can select FOVs of interest as well

In [13]:
sdata.sel(elements=["fov0", "fov8"])

SpatialData object with:
└── Images
      ├── 'fov0': SpatialImage[cyx] (22, 512, 512)
      └── 'fov8': SpatialImage[cyx] (22, 1024, 1024)
with coordinate systems:
▸ 'global', with elements:
        fov0 (Images), fov8 (Images)

We can broadcast a query across axes for certain coordinates too

In [15]:
sdata.query.bounding_box(
    axes=["x", "y"],
    min_coordinate=[0, 0],
    max_coordinate=[256, 256],
    target_coordinate_system="global",
)

SpatialData object with:
└── Images
      ├── 'fov0': SpatialImage[cyx] (22, 256, 256)
      ├── 'fov1': SpatialImage[cyx] (22, 256, 256)
      ├── 'fov2': SpatialImage[cyx] (22, 256, 256)
      ├── 'fov3': SpatialImage[cyx] (22, 256, 256)
      ├── 'fov4': SpatialImage[cyx] (22, 256, 256)
      ├── 'fov5': SpatialImage[cyx] (22, 256, 256)
      ├── 'fov6': SpatialImage[cyx] (22, 256, 256)
      ├── 'fov7': SpatialImage[cyx] (22, 256, 256)
      ├── 'fov8': SpatialImage[cyx] (22, 256, 256)
      ├── 'fov9': SpatialImage[cyx] (22, 256, 256)
      └── 'fov10': SpatialImage[cyx] (22, 256, 256)
with coordinate systems:
▸ 'global', with elements:
        fov0 (Images), fov1 (Images), fov2 (Images), fov3 (Images), fov4 (Images), fov5 (Images), fov6 (Images), fov7 (Images), fov8 (Images), fov9 (Images), fov10 (Images)

Lets save the Spatial Data object as a OME ZARR Store

In [ ]:
cohort_sd_save_path = Path("../data/cohorts/example_cohort.ome.zarr")

In [ ]:
sdata.write(file_path=cohort_sd_save_path)

We can load the Zarr store to a spatial data object as well.

In [ ]:
sdata2 = sd.read_zarr(store=cohort_sd_save_path)